In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics

In [8]:
# transform method 1
tmp_train1 = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transforms.ToTensor())

# trasform method 2
transform = transforms.Compose([transforms.ToTensor()])
tmp_train2 = torchvision.datasets.MNIST(root='./data',train=True, download=True, transform=transform)

In [10]:
print(tmp_train1[0])
print(tmp_train2[0])

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [2]:
class ConvModel(nn.Module):
  def __init__(self):
    super().__init__()

    # image size = 28*28*1
    self.conv = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
    self.lin1 = nn.Linear(26*26*32, 128)
    self.lin2 = nn.Linear(128, 10)

  def forward(self, x):
    # batch size = 32
    # 32 * 1 * 28 * 28 -> 32 * 32 * 26 * 26
    x = self.conv(x)
    x = F.relu(x)

    x = x.flatten(start_dim = 1)
    
    x = self.lin1(x)
    x = F.relu(x)

    x = self.lin2(x)
    out = F.softmax(x, dim=1)
    return out

In [28]:
import numpy as np

a = np.array([[[1, 2],[3, 4]],[[5, 6],[7, 8]]])
b = np.arange(3*2*4).reshape((3,2,4))

ta = torch.tensor(a)
tb = torch.tensor(b)

print(f'original tensor shape = {ta.shape}\n {ta}\n')

ta_f0 = ta.flatten(start_dim=0)
print(f'flatten ta shape = {ta_f0.shape} (start_dim = 0) \n {ta_f0}\n')

ta_f1 = ta.flatten(start_dim=1)
print(f'flatten ta shape = {ta_f1.shape} (start_dim = 1) \n {ta_f1}\n')

ta_f2 = ta.flatten(start_dim=2)
print(f'flatten ta shape = {ta_f2.shape} (start_dim = 2) \n {ta_f2}\n')

print("---------------------------------------------")
print(f'original tensor shape = {tb.shape}\n {tb}\n')

tb_f0 = tb.flatten(start_dim=0)
print(f'flatten tb shape = {tb_f0.shape} (start_dim = 0) \n {tb_f0}\n')

tb_f1 = tb.flatten(start_dim=1)
print(f'flatten tb shape = {tb_f1.shape} (start_dim = 1) \n {tb_f1}\n')

tb_f2 = tb.flatten(start_dim=2)
print(f'flatten tb shape = {tb_f2.shape} (start_dim = 2) \n {tb_f2}\n')

original tensor shape = torch.Size([2, 2, 2])
 tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])

flatten ta shape = torch.Size([8]) (start_dim = 0) 
 tensor([1, 2, 3, 4, 5, 6, 7, 8])

flatten ta shape = torch.Size([2, 4]) (start_dim = 1) 
 tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])

flatten ta shape = torch.Size([2, 2, 2]) (start_dim = 2) 
 tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])

---------------------------------------------
original tensor shape = torch.Size([3, 2, 4])
 tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7]],

        [[ 8,  9, 10, 11],
         [12, 13, 14, 15]],

        [[16, 17, 18, 19],
         [20, 21, 22, 23]]])

flatten tb shape = torch.Size([24]) (start_dim = 0) 
 tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23])

flatten tb shape = torch.Size([3, 8]) (start_dim = 1) 
 tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 8,  9, 10, 11, 12, 13

In [3]:
BATCH_SIZE = 32

train_data = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

test_data = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [35]:
print(len(train_data))
print(len(trainloader))
print(1875*32)

60000
1875
60000


In [4]:
l_rate = 0.001
num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = ConvModel()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = l_rate)

In [5]:
device

device(type='cuda', index=0)

In [6]:
for ep in range(num_epochs):
  train_loss = 0.0
  train_acc = 0.0

  for i, (imgs, labs) in enumerate(trainloader):
    imgs = imgs.to(device)
    labs = labs.to(device)

    out = model(imgs)
    loss = criterion(out, labs)
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    train_loss += loss.detach().item()
    train_acc += (torch.argmax(out, 1).flatten() == labs).type(torch.float).mean().item()

  print(f"[Epoch {ep}] Loss : {train_loss/i:.4f} / Accuracy : {train_acc/i:.4f}")

[Epoch 0] Loss : 2912.7861 / Accuracy : 1711.2188
[Epoch 1] Loss : 2796.4710 / Accuracy : 1822.3125
[Epoch 2] Loss : 2779.6698 / Accuracy : 1837.6875
[Epoch 3] Loss : 2770.1961 / Accuracy : 1846.2812
[Epoch 4] Loss : 2765.2052 / Accuracy : 1850.4062


In [9]:
test_acc = 0.0

for i, (imgs, labs) in enumerate(testloader):
  imgs = imgs.to(device)
  labs = labs.to(device)

  outputs = model(imgs)
  test_acc += (torch.argmax(outputs, 1).flatten() == labs).type(torch.float).mean().item()
  preds = torch.argmax(outputs, 1).flatten().cpu().numpy()
print(f'test Accuracy : {test_acc/i:.4f}')

test Accuracy : 0.9856
